In [1]:
import numpy as np
import pandas as pd
#import _pickle as cPickle # python3 shit
import re, sys, os, csv
from many_stop_words import get_stop_words
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#from itertools import chain
#from collections import defaultdict
from collections import Counter

In [2]:
#stop_words = list(get_stop_words('en'))         #About 900 stop words
#nltk_words = list(stopwords.words('english'))   #About 150 stop words
#stop_words.extend(nltk_words)

def word_prob(word): return dictionary[word] / total
def words(text): return re.findall('[a-z]+', text.lower())
dictionary = Counter(words(open('20k.txt').read()))
max_word_length = max(map(len, dictionary))
total = float(sum(dictionary.values()))

def viterbi_segment(text):
    probs, lasts = [1.0], [0]
    for i in range(1, len(text) + 1):
        prob_k, k = max((probs[j] * word_prob(text[j:i]), j)
                        for j in range(max(0, i - max_word_length), i))
        probs.append(prob_k)
        lasts.append(k)
    words = []
    i = len(text)
    while 0 < i:
        words.append(text[lasts[i]:i])
        i = lasts[i]
    words.reverse()
    return words, probs[-1]

def fix_hashtag(text):
    print(text, end = " ")
    text = text.group().split(":")[0]
    text = text[1:] # remove '#'
    output = ' '.join(viterbi_segment(text)[0])
    print(output)
    return output

def clean_tweet( tweet):
        tweet = tweet.lower()
        tweet = re.sub("(#[A-Za-z0-9]+)", fix_hashtag, tweet)
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

def remove_stopwords(word_list):
        filtered_tweet=""
        for word in word_list:
            word = word.lower() 
            if word not in stopwords.words("english"):
                filtered_tweet=filtered_tweet + " " + word
        
        return filtered_tweet.lstrip()
    
def vectorise_label(label):
    if label == "empty":return 0
    elif label == "sadness":return 1
    elif label == "enthusiasm":return 2
    elif label == "neutral":return 0
    elif label == "worry":return 1
    elif label == "surprise":return 2
    elif label == "love":return 2
    elif label == "fun":return 2
    elif label == "hate":return 3
    elif label == "happiness":return 2
    elif label == "boredom":return 0
    elif label == "relief":return 2
    elif label == "anger":return 3

In [3]:
data_train = pd.read_csv('dataset/data/text_emotion.csv', sep=',')
print("Dataset shape:",data_train.shape)
print(data_train.sentiment[0],":",data_train.content[0])

Dataset shape: (40000, 4)
empty : @tiffanylue i know  i was listenin to bad habit earlier and i started freakin at his part =[


In [4]:
dataWriter = csv.writer(open('data_clean.csv', 'w'), delimiter=',',lineterminator="\n")

total = 40000
for i in range(40000):
    #print("Progress: ",round(i/total*100,2),"   ",end="\r")
    tweet= clean_tweet(data_train.content[i])
    #tweet = remove_stopwords(tweet.split())
    dataWriter.writerow([tweet, str(vectorise_label(data_train.sentiment[i]))])
    #sys.stdout.write("\033[F")
    
print("Progress: ",100,"\nComplete!")

<_sre.SRE_Match object; span=(110, 120), match='#topicmaps'> topic maps
<_sre.SRE_Match object; span=(67, 71), match='#uds'> ud s
<_sre.SRE_Match object; span=(114, 117), match='#fb'> fb
<_sre.SRE_Match object; span=(0, 15), match='#3wordsaftersex'> 3 w o r d s a f t e r s e x
<_sre.SRE_Match object; span=(0, 14), match='#liesgirlstell'> lies girls tell
<_sre.SRE_Match object; span=(0, 5), match='#beer'> beer
<_sre.SRE_Match object; span=(6, 13), match='#friday'> friday
<_sre.SRE_Match object; span=(14, 24), match='#melbourne'> melbourne
<_sre.SRE_Match object; span=(25, 29), match='#cbd'> cbd
<_sre.SRE_Match object; span=(30, 35), match='#mtub'> mt ub
<_sre.SRE_Match object; span=(36, 41), match='#mini'> mini
<_sre.SRE_Match object; span=(42, 47), match='#cmon'> cm on
<_sre.SRE_Match object; span=(0, 13), match='#jonaswebcast'> jonas webcast
<_sre.SRE_Match object; span=(79, 85), match='#lupus'> lu pu s
<_sre.SRE_Match object; span=(74, 80), match='#linux'> linux
<_sre.SRE_Match objec

<_sre.SRE_Match object; span=(20, 23), match='#fb'> fb
<_sre.SRE_Match object; span=(108, 126), match='#fridayfirehazzard'> friday fire ha zz ar d
<_sre.SRE_Match object; span=(80, 86), match='#asylm'> as y lm
<_sre.SRE_Match object; span=(24, 29), match='#eyes'> eyes
<_sre.SRE_Match object; span=(31, 36), match='#twpp'> twp p
<_sre.SRE_Match object; span=(37, 40), match='#fb'> fb
<_sre.SRE_Match object; span=(92, 98), match='#metal'> metal
<_sre.SRE_Match object; span=(22, 38), match='#jrztwitterlunch'> jr z tw it ter lunch
<_sre.SRE_Match object; span=(25, 34), match='#geocache'> geo cache
<_sre.SRE_Match object; span=(134, 137), match='#fb'> fb
<_sre.SRE_Match object; span=(74, 81), match='#rstats'> r stats
<_sre.SRE_Match object; span=(60, 68), match='#itsucks'> it sucks
<_sre.SRE_Match object; span=(74, 82), match='#itsucks'> it sucks
<_sre.SRE_Match object; span=(27, 40), match='#followfriday'> follow friday
<_sre.SRE_Match object; span=(97, 107), match='#rrtheatre'> rr theatre
<

<_sre.SRE_Match object; span=(88, 92), match='#bgt'> bg t
<_sre.SRE_Match object; span=(0, 10), match='#justcause'> just cause
<_sre.SRE_Match object; span=(33, 36), match='#fb'> fb
<_sre.SRE_Match object; span=(52, 56), match='#bgt'> bg t
<_sre.SRE_Match object; span=(55, 68), match='#followfriday'> follow friday
<_sre.SRE_Match object; span=(26, 30), match='#bgt'> bg t
<_sre.SRE_Match object; span=(49, 62), match='#humpthestump'> hum pt hes tu mp
<_sre.SRE_Match object; span=(0, 15), match='#3wordsaftersex'> 3 w o r d s a f t e r s e x
<_sre.SRE_Match object; span=(130, 134), match='#wow'> wow
<_sre.SRE_Match object; span=(37, 40), match='#ff'> ff
<_sre.SRE_Match object; span=(41, 45), match='#bgt'> bg t
<_sre.SRE_Match object; span=(69, 77), match='#pughugs'> pu g hugs
<_sre.SRE_Match object; span=(118, 123), match='#twpp'> twp p
<_sre.SRE_Match object; span=(0, 4), match='#bgt'> bg t
<_sre.SRE_Match object; span=(90, 96), match='#g4c09'> g 4 c 0 9
<_sre.SRE_Match object; span=(43, 

<_sre.SRE_Match object; span=(22, 32), match='#hoppusday'> hop p usda y
<_sre.SRE_Match object; span=(74, 84), match='#hoppusday'> hop p usda y
<_sre.SRE_Match object; span=(104, 114), match='#hoppusday'> hop p usda y
<_sre.SRE_Match object; span=(23, 31), match='#twitter'> tw it ter
<_sre.SRE_Match object; span=(26, 31), match='#jaoo'> ja oo
<_sre.SRE_Match object; span=(53, 64), match='#edumedia09'> edu media 0 9
<_sre.SRE_Match object; span=(111, 119), match='#nethack'> net hack
<_sre.SRE_Match object; span=(27, 40), match='#followfriday'> follow friday
<_sre.SRE_Match object; span=(6, 14), match='#juddday'> judd day
<_sre.SRE_Match object; span=(100, 112), match='#musicmonday'> music monday
<_sre.SRE_Match object; span=(78, 84), match='#idiot'> idiot
<_sre.SRE_Match object; span=(52, 65), match='#followfriday'> follow friday
<_sre.SRE_Match object; span=(9, 18), match='#swineflu'> s wine flu
<_sre.SRE_Match object; span=(43, 47), match='#toe'> toe
<_sre.SRE_Match object; span=(102,

<_sre.SRE_Match object; span=(0, 6), match='#happy'> happy
<_sre.SRE_Match object; span=(85, 104), match='#productnamingrulez'> product naming rule z
<_sre.SRE_Match object; span=(41, 45), match='#npr'> npr
<_sre.SRE_Match object; span=(100, 107), match='#monday'> monday
<_sre.SRE_Match object; span=(46, 55), match='#roidweek'> roi d week
<_sre.SRE_Match object; span=(81, 94), match='#photofollows'> photo follows
<_sre.SRE_Match object; span=(76, 79), match='#fb'> fb
<_sre.SRE_Match object; span=(107, 119), match='#starwarsday'> star wars day
<_sre.SRE_Match object; span=(15, 25), match='#dollhouse'> doll house
<_sre.SRE_Match object; span=(51, 69), match='#hugecoffeesuccess'> huge coffee success
<_sre.SRE_Match object; span=(15, 17), match='#1'> 1
<_sre.SRE_Match object; span=(79, 85), match='#movie'> movie
<_sre.SRE_Match object; span=(3, 15), match='#musicmonday'> music monday
<_sre.SRE_Match object; span=(74, 86), match='#starwarsday'> star wars day
<_sre.SRE_Match object; span=(87

<_sre.SRE_Match object; span=(120, 127), match='#twitip'> tw it ip
<_sre.SRE_Match object; span=(125, 140), match='#openhacklondon'> open hack london
<_sre.SRE_Match object; span=(14, 24), match='#followers'> followers
<_sre.SRE_Match object; span=(39, 44), match='#pane'> pane
<_sre.SRE_Match object; span=(6, 8), match='#7'> 7
<_sre.SRE_Match object; span=(0, 5), match='#test'> test
<_sre.SRE_Match object; span=(32, 39), match='#iphone'> i phone
<_sre.SRE_Match object; span=(63, 65), match='#1'> 1
<_sre.SRE_Match object; span=(27, 36), match='#britains'> britain s
<_sre.SRE_Match object; span=(82, 93), match='#bigweekend'> big weekend
<_sre.SRE_Match object; span=(0, 6), match='#web20'> web 2 0
<_sre.SRE_Match object; span=(8, 17), match='#prologue'> pro log ue
<_sre.SRE_Match object; span=(93, 96), match='#fb'> fb
<_sre.SRE_Match object; span=(68, 72), match='#fml'> fm l
<_sre.SRE_Match object; span=(16, 28), match='#warmfuzzies'> warm fu zz ies
<_sre.SRE_Match object; span=(107, 110)

In [12]:
dataWriter = csv.writer(open('data_clean.csv', 'w'), delimiter=',',lineterminator="\n")

total = 40000
for i in range(40000):
    print("Progress: ",round(i/total*100,2),"   ",end="\r")
    tweet= clean_tweet(data_train.content[i])
    #tweet = remove_stopwords(tweet.split())
    dataWriter.writerow([tweet, str(vectorise_label(data_train.sentiment[i]))])
    sys.stdout.write("\033[F")
    
print("Progress: ",100,"\nComplete!")

<_sre.SRE_Match object; span=(110, 120), match='#topicmaps'> topic maps
<_sre.SRE_Match object; span=(67, 71), match='#uds'> ud s
<_sre.SRE_Match object; span=(114, 117), match='#fb'> fb
<_sre.SRE_Match object; span=(0, 15), match='#3wordsaftersex'> 3 w o r d s a f t e r s e x


<_sre.SRE_Match object; span=(0, 14), match='#liesgirlstell'> lies girls tell
<_sre.SRE_Match object; span=(0, 5), match='#beer'> beer
<_sre.SRE_Match object; span=(6, 13), match='#friday'> friday
<_sre.SRE_Match object; span=(14, 24), match='#melbourne'> melbourne
<_sre.SRE_Match object; span=(25, 29), match='#cbd'> cbd
<_sre.SRE_Match object; span=(30, 35), match='#mtub'> mt ub
<_sre.SRE_Match object; span=(36, 41), match='#mini'> mini
<_sre.SRE_Match object; span=(42, 47), match='#cmon'> cm on
<_sre.SRE_Match object; span=(0, 13), match='#jonaswebcast'> jonas webcast
<_sre.SRE_Match object; span=(79, 85), match='#lupus'> lu pu s
<_sre.SRE_Match object; span=(74, 80), match='#linux'> linux
<_sre.SRE_Match object; span=(81, 88), match='#ubuntu'> ubuntu


<_sre.SRE_Match object; span=(119, 121), match='#s'> s
<_sre.SRE_Match object; span=(40, 42), match='#5'> 5
<_sre.SRE_Match object; span=(42, 48), match='#yahoo'> yahoo
<_sre.SRE_Match object; span=(57, 59), match='#1'> 1
<_sre.SRE_Match object; span=(0, 16), match='#thingsmummysaid'> things mummy said


<_sre.SRE_Match object; span=(88, 95), match='#dsaa09'> dsa a 0 9
<_sre.SRE_Match object; span=(84, 87), match='#fb'> fb
<_sre.SRE_Match object; span=(102, 111), match='#windows7'> windows 7
<_sre.SRE_Match object; span=(0, 12), match='#depechemode'> depeche mode
<_sre.SRE_Match object; span=(69, 80), match='#boredboots'> bored boots
<_sre.SRE_Match object; span=(85, 90), match='#fail'> fail
<_sre.SRE_Match object; span=(38, 43), match='#eric'> eric
<_sre.SRE_Match object; span=(0, 15), match='#3wordsaftersex'> 3 w o r d s a f t e r s e x
<_sre.SRE_Match object; span=(124, 127), match='#fb'> fb


<_sre.SRE_Match object; span=(17, 28), match='#nightshift'> night shift
<_sre.SRE_Match object; span=(54, 63), match='#badoptus'> bad opt us
<_sre.SRE_Match object; span=(9, 20), match='#googledocs'> google docs
<_sre.SRE_Match object; span=(56, 62), match='#gmail'> gmail
<_sre.SRE_Match object; span=(123, 135), match='#warisfucked'> war is fucked
<_sre.SRE_Match object; span=(67, 73), match='#perth'> perth


...

<_sre.SRE_Match object; span=(44, 60), match='#twittertakeover'> tw it ter takeover
<_sre.SRE_Match object; span=(0, 3), match='#f1'> f 1
<_sre.SRE_Match object; span=(24, 27), match='#fb'> fb
<_sre.SRE_Match object; span=(44, 47), match='#ff'> ff
<_sre.SRE_Match object; span=(90, 102), match='#houseofrock'> house of rock
<_sre.SRE_Match object; span=(41, 57), match='#twittertakeover'> tw it ter takeover
<_sre.SRE_Match object; span=(63, 66), match='#f1'> f 1
<_sre.SRE_Match object; span=(30, 42), match='#ffrecommend'> ff recommend
<_sre.SRE_Match object; span=(90, 93), match='#fb'> fb
<_sre.SRE_Match object; span=(101, 107), match='#logic'> logic
<_sre.SRE_Match object; span=(38, 54), match='#twittertakeover'> tw it ter takeover
<_sre.SRE_Match object; span=(37, 47), match='#netplayer'> net player
<_sre.SRE_Match object; span=(0, 16), match='#sanctuarysunday'> sanctuary sunday
<_sre.SRE_Match object; span=(36, 43), match='#iphone'> i phone
<_sre.SRE_Match object; span=(42, 52), match=

<_sre.SRE_Match object; span=(13, 21), match='#all4web'> all 4 w e b
<_sre.SRE_Match object; span=(0, 10), match='#duckraces'> duck races
<_sre.SRE_Match object; span=(105, 109), match='#ing'> ing
<_sre.SRE_Match object; span=(63, 73), match='#matternal'> matter na l
<_sre.SRE_Match object; span=(0, 16), match='#sanctuarysunday'> sanctuary sunday
<_sre.SRE_Match object; span=(9, 14), match='#stud'> stud
<_sre.SRE_Match object; span=(83, 89), match='#dates'> dates
<_sre.SRE_Match object; span=(61, 77), match='#urbanoutfitters'> urban outfitters
<_sre.SRE_Match object; span=(20, 27), match='#euruko'> eur uk o
<_sre.SRE_Match object; span=(73, 78), match='#dsds'> ds ds


<_sre.SRE_Match object; span=(84, 92), match='#ableton'> able ton
<_sre.SRE_Match object; span=(46, 50), match='#yao'> ya o
<_sre.SRE_Match object; span=(77, 85), match='#rockets'> rockets
<_sre.SRE_Match object; span=(94, 99), match='#kobe'> kobe
<_sre.SRE_Match object; span=(95, 106), match='#youregreat'> your e great
<_sre.SRE_Match object; span=(52, 67), match='#maternalhealth'> maternal health
<_sre.SRE_Match object; span=(28, 39), match='#springtime'> spring time
<_sre.SRE_Match object; span=(27, 30), match='#fb'> fb
<_sre.SRE_Match object; span=(25, 36), match='#masterchef'> master chef
<_sre.SRE_Match object; span=(46, 57), match='#nightshift'> night shift
<_sre.SRE_Match object; span=(17, 21), match='#run'> run
<_sre.SRE_Match object; span=(14, 26), match='#squarespace'> squares pace
<_sre.SRE_Match object; span=(84, 95), match='#masterchef'> master chef
<_sre.SRE_Match object; span=(95, 106), match='#mothersday'> mothers day


<_sre.SRE_Match object; span=(25, 32), match='#euruko'> eur uk o
<_sre.SRE_Match object; span=(0, 16), match='#sanctuarysunday'> sanctuary sunday
<_sre.SRE_Match object; span=(35, 48), match='#localgovcamp'> local gov camp
<_sre.SRE_Match object; span=(55, 62), match='#euruko'> eur uk o
<_sre.SRE_Match object; span=(9, 12), match='#f1'> f 1
<_sre.SRE_Match object; span=(96, 101), match='#lost'> lost
<_sre.SRE_Match object; span=(11, 25), match='#wheniwerealad'> when i were ala d
<_sre.SRE_Match object; span=(0, 8), match='#firefly'> firefly
<_sre.SRE_Match object; span=(9, 21), match='#barcampbkk3'> bar camp bk k 3
<_sre.SRE_Match object; span=(69, 76), match='#euruko'> eur uk o


Progress:  100 
Complete!


In [13]:
count = 0
with open('data_clean.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        count+=1

In [14]:
count

40000